## Thesis - Clean Books Text

**Authors:** Sara Caicedo  
**Implementation:** Sara Caicedo     
**Email:** s.caicedos2@uniandes.edu.co  
**Dependencies:**  
**Created:** August 27 - 2025 

**Last Modified:** August 27 - 2025 
**Version:** 01  

**Input:** PDFs in , `Tesis/input/04.Chi2`

**Output:** txt in , `Tesis/output/Chi2`

### Description
1. Clean the PDF formats into txt \
This is a one time thing, just do it


In [1]:
!pip install pypdf
!pip install nltk


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
## Download Packages
import subprocess
import sys
import fitz  # PyMuPDF
import pandas as pd
import re
from pypdf import PdfReader
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.punkt import PunktSentenceTokenizer

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Install Packages
install("pymupdf")
install("sqlalchemy")
install("pandas")

/opt/anaconda3/envs/thesis_scripts/lib/python3.12/site-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


# 00. Create the functions to clean and process the text from the series and the book

## 0.1 If needed, crop the pdf so that the headers are not visible to the pdf reader

In [ ]:
from PyPDF2 import PdfReader, PdfWriter

def crop_pdf(input_pdf, output_pdf, left=50, bottom=50, right=50, top=50):
    """
    Crop margins from all pages of a PDF.
    left, bottom, right, top = number of points (1 pt = 1/72 inch)
    """
    reader = PdfReader(input_pdf)
    writer = PdfWriter()

    for page in reader.pages:
        # Get original dimensions
        box = page.mediabox
        x0, y0, x1, y1 = float(box.lower_left[0]), float(box.lower_left[1]), float(box.upper_right[0]), float(box.upper_right[1])

        # Define new cropped box
        new_x0 = x0 + left
        new_y0 = y0 + bottom
        new_x1 = x1 - right
        new_y1 = y1 - top

        # Apply crop
        page.mediabox.lower_left = (new_x0, new_y0)
        page.mediabox.upper_right = (new_x1, new_y1)

        writer.add_page(page)

    with open(output_pdf, "wb") as f:
        writer.write(f)

# Example usage
og_file = "/Users/saracaicedo/Library/CloudStorage/OneDrive-Universidaddelosandes/2025-1/Tesis/input/04.Chi2/AtlasShrugged_AynRand.pdf"
bx_file = "/Users/saracaicedo/Library/CloudStorage/OneDrive-Universidaddelosandes/2025-1/Tesis/input/04.Chi2/Clean_AtlasShrugged_AynRand.pdf"
crop_pdf(og_file, bx_file, left=20, bottom=30, right=20, top=70)


## 0.2 Extract the data from the book and clean the regular expressions that are not necessary 

In [85]:
doc = fitz.open("/Users/saracaicedo/Library/CloudStorage/OneDrive-Universidaddelosandes/2025-1/Tesis/input/04.Chi2/1984_Orwell.pdf")
for page in doc:
    page_text_raw = page.get_text()

In [86]:
page_text_raw

'It would have been quite impossible to render this into Newspeak while\nkeeping to the sense of the original. The nearest one could come to doing so\nwould be to swallow the whole passage up in the single word crimethink. A full\ntranslation could only be an ideological translation, whereby Jeﬀerson’s words\nwould be changed into a panegyric on absolute government.\nA good deal of the literature of the past was, indeed, already being trans-\nformed in this way. Considerations of prestige made it desirable to preserve\nthe memory of certain historical ﬁgures, while at the same time bringing their\nachievements into line with the philosophy of Ingsoc. Various writers, such as\nShakespeare, Milton, Swift, Byron, Dickens, and some others were therefore in\nprocess of translation: when the task had been completed, their original writ-\nings, with all else that survived of the literature of the past, would be destroyed.\nThese translations were a slow and diﬃcult business, and it was not ex

In [87]:
# This funtion returns a document, not the string, that is why I still have to transform it into a string later
def extract_words(pdf_path, txt_path):
    # Regular expression pattern to match the dynamic header
    #title_pattern = r"FREE TO CHOOSE: A Personal Statement"

    # Regular expression pattern to match footer text indicating page information
    chapter_pattern = r"C H A P T E R|●"
    
    # Numbers to be deleted
    number_pattern = r"\b(?:[1-9]|[1-9][0-9]|1[0-9]{2}|2[0-9]{2}|3[0-9]{2}|314)\b\n"

    # Open the PDF file
    doc = fitz.open(pdf_path)
    print("this is the doc:", doc)
    # Iterate through each page of the PDF
    with open(txt_path, "w") as txt_file:
        for page in doc:
            page_text_raw = page.get_text()

            # Use regular expression to remove book title text
            #page_text = re.sub(title_pattern, "", page_text, flags=re.DOTALL)

            # Use regular expression to remove chapter text
            page_text = re.sub(chapter_pattern, "", page_text_raw)
            
            # Delete the numbers of the pages
            page_text = re.sub(number_pattern, "", page_text)

            # Delete the -
            page_text = re.sub(r'-\n', "", page_text)
        
            # Replace the "fi " and "fl " in words like "infl ation", "defi nition"
            #page_text = re.sub(r"fi\s", "fi", page_text)
        
            #page_text = re.sub(r"fl\s", "fl", page_text)

            # Add the text to the txt file
            txt_file.write(page_text)
            txt_file.write("\n")

            print(page_text_raw)

    # Close the document
    doc.close()

In [88]:
pdf_path = r"/Users/saracaicedo/Library/CloudStorage/OneDrive-Universidaddelosandes/2025-1/Tesis/input/04.Chi2/1984_Orwell.pdf"
txt_path = r'/Users/saracaicedo/Library/CloudStorage/OneDrive-Universidaddelosandes/2025-1/Tesis/output/Chi2/1984_Orwell.txt'
txt_series_path = r'/Users/saracaicedo/Library/CloudStorage/OneDrive-Universidaddelosandes/2025-1/Tesis/output/friedman_txt/Series_Free to Choose.txt'
# Extract words, marking line jumps, and create CSV
extract_words(pdf_path, txt_path)

this is the doc: Document('/Users/saracaicedo/Library/CloudStorage/OneDrive-Universidaddelosandes/2025-1/Tesis/input/04.Chi2/1984_Orwell.pdf')
1984
George Orwell
1949

Chapter 1
It was a bright cold day in April, and the clocks were striking thirteen. Winston
Smith, his chin nuzzled into his breast in an eﬀort to escape the vile wind, slipped
quickly through the glass doors of Victory Mansions, though not quickly enough
to prevent a swirl of gritty dust from entering along with him.
The hallway smelt of boiled cabbage and old rag mats. At one end of it a
coloured poster, too large for indoor display, had been tacked to the wall. It
depicted simply an enormous face, more than a metre wide: the face of a man of
about forty-ﬁve, with a heavy black moustache and ruggedly handsome features.
Winston made for the stairs. It was no use trying the lift. Even at the best
of times it was seldom working, and at present the electric current was cut oﬀ
during daylight hours. It was part of the econo

## Tokenize and lemmatize the text

In [63]:
# Add congressional-specific stopwords
Friedman_stopwords = {
    "would", "century", "would"
}
# Set of English stopwords
stop_words = set(stopwords.words('english'))
# Combine with standard stopwords
stop_words.update(Friedman_stopwords)

In [64]:
def preprocess_text(text):
    # Remove numbers and special characters
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    # Convert to lowercase
    text = text.lower()

    # Tokenize the text
    tokens = word_tokenize(text)

    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]

    # Lemmatization: grouping together different inflected forms of the same word
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    #print(tokens[:50])
    print(tokens)
    return tokens

In [66]:
def preprocess_text_video(text):
    # Remove numbers, text in parentheses, speaker labels, and punctuation
    text = re.sub(r'\d+', '', text)                    # Remove numbers
    text = re.sub(r'\([^)]*\)', '', text)              # Remove parentheses and content inside
    text = re.sub(r'\b\w+:\s*', '', text)              # Remove speaker labels like "John:"
    text = re.sub(r'[^\w\s]', '', text)                # Remove punctuation

    # Normalize case and trim
    text = text.lower().strip()

    # Tokenize
    tokens = word_tokenize(text)

    # Optional: remove stopwords
    tokens = [word for word in tokens if word not in stop_words]

    # Optional: lemmatize
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return tokens


# 1. Get n-grams for the book

In [89]:
# Open the txt of the book
from itertools import chain
with open(txt_path, 'rb') as file:
    text_string = file.read().decode('utf-8', errors='ignore')  # Ignora caracteres problemáticos
# Create chunks of text
chunks = text_string.split('.')
# Eliminate the withespaces

chunks = [chunk.strip() for chunk in chunks if chunk.strip()]

# Clean each chunk
clean_all = []
for chunk in chunks:
    clean_chunk = preprocess_text(chunk)
    clean_all.append(clean_chunk)

# Flatten the list of lists into a single list of tokens
clean_txt = list(chain.from_iterable(clean_all))


['ever', 'since', 'first', 'settlement', 'european', 'new', 'world', 'america', 'magnet', 'people', 'seeking', 'adventure', 'fleeing', 'tyranny', 'simply', 'trying', 'make', 'better', 'life', 'child']
['initial', 'trickle', 'swelled', 'american', 'revolution', 'establishment', 'united', 'state', 'america', 'became', 'flood', 'nineteenth', 'million', 'people', 'streamed', 'across', 'atlantic', 'smaller', 'number', 'across', 'pacific', 'driven', 'misery', 'tyranny', 'attracted', 'promise', 'freedom', 'affluence']
['arrived', 'find', 'street', 'paved', 'gold', 'find', 'easy', 'life']
['find', 'freedom', 'opportunity', 'make', 'talent']
['hard', 'work', 'ingenuity', 'thrift', 'luck', 'succeeded', 'realizing', 'enough', 'hope', 'dream', 'encourage', 'friend', 'relative', 'join']
['story', 'united', 'state', 'story', 'economic', 'miracle', 'political', 'miracle', 'made', 'possible', 'translation', 'practice', 'two', 'set', 'ideasboth', 'curious', 'coincidence', 'formulated', 'document', 'pub

## Analyze the book


In [ ]:
from collections import Counter
from nltk.util import ngrams

In [ ]:
from nltk.util import ngrams
texto_f = [w for w in clean_txt]

list(ngrams(clean_txt, 3))[0:30] #Frecuencia de palabras que vienen juntas

[('vol', 'hello', 'im'),
 ('hello', 'im', 'robert'),
 ('im', 'robert', 'mckenzie'),
 ('robert', 'mckenzie', 'join'),
 ('mckenzie', 'join', 'fine'),
 ('join', 'fine', 'old'),
 ('fine', 'old', 'library'),
 ('old', 'library', 'campus'),
 ('library', 'campus', 'university'),
 ('campus', 'university', 'chicago'),
 ('university', 'chicago', 'see'),
 ('chicago', 'see', 'first'),
 ('see', 'first', 'films'),
 ('first', 'films', 'milton'),
 ('films', 'milton', 'friedman'),
 ('milton', 'friedman', 'economist'),
 ('friedman', 'economist', 'international'),
 ('economist', 'international', 'repute'),
 ('international', 'repute', 'winner'),
 ('repute', 'winner', 'nobel'),
 ('winner', 'nobel', 'prize'),
 ('nobel', 'prize', 'one'),
 ('prize', 'one', 'controversial'),
 ('one', 'controversial', 'stimulating'),
 ('controversial', 'stimulating', 'americans'),
 ('stimulating', 'americans', 'time'),
 ('americans', 'time', 'weeks'),
 ('time', 'weeks', 'lie'),
 ('weeks', 'lie', 'ahead'),
 ('lie', 'ahead', 'fri

## N-grams

In [90]:
# Función para determinar el orden del n-grama
def top_k_ngrams(word_tokens, n, k, stopword_phrases=None):
    """
    Parameters:
    - word_tokens: list of tokens (already preprocessed)
    - n: n-gram size (e.g., 2 for bigrams)
    - k: number of top n-grams to return
    - stopword_phrases: set of phrases (strings) to exclude (e.g., {'united states'})
    """

    # Crear lista de n-gramas
    n_gram_list = list(ngrams(word_tokens, n))

    # Convertir cada n-grama a un string
    n_gram_strings = [' '.join(gram) for gram in n_gram_list]

    # Filtrar frases stopword si se pasan como argumento
    if stopword_phrases is not None:
        n_gram_strings = [phrase for phrase in n_gram_strings if phrase not in stopword_phrases]

    # Contar frecuencia y devolver los k más comunes
    n_gram_counter = Counter(n_gram_strings)
    return n_gram_counter.most_common(k)

In [91]:
stop_phrases = {"nineteenth century", 'one another','new york', 'tide l turning', 'tide l turning', 'ever bigger government','federal reserve board'}

# Bi-grama
bigram = top_k_ngrams(clean_txt,2,20, stopword_phrases=stop_phrases)
bigram_list= list(bigram)

# Bi-grama
trigram = top_k_ngrams(clean_txt,3,20, stopword_phrases=stop_phrases)
trigram_list= list(trigram)
####### Create latex table with the words #######
# LaTeX table header

latex_table = r"""
\begin{threeparttable}
\begin{tabular}{l l c l c}\\

\multicolumn{2}{c}{\textbf{Bigrams}}& & \multicolumn{2}{c}{\textbf{Trigrams}} \\
\cline{1-2} \cline{4-5} \textbf{Phrase} & \textbf{Count} & & \textbf{Phrase} & \textbf{Count}\\
\hline
\hline
"""

# Iterate over the list and add rows to the LaTeX table
for biagram, trigram in zip(bigram_list, trigram_list):
    phrase_b = biagram[0]
    count_b = biagram[1]

    phrase_t = trigram[0]
    count_t = trigram[1]
    
    latex_table += f"{phrase_b} & {count_b} & & {phrase_t} & {count_t} \\\\\n"

# LaTeX table footer
latex_table += r""" \hline
\hline
\end{tabular} \begin{tablenotes} 
\item{\footnotesize \tablenote} \end{tablenotes} \end{threeparttable}
"""

# Save the LaTeX table to a .txt file
with open("/Users/saracaicedo/Library/CloudStorage/OneDrive-Universidaddelosandes/2025-1/Tesis/output/tables/book_ngrams.tex", "w") as file:
    file.write(latex_table)

In [61]:
stop_phrases = {"nineteenth century", 'one another','new york', 'tide l turning', 'tide l turning', 'ever bigger government','federal reserve board'}
# Tri-grama
Trigram = top_k_ngrams(clean_txt,3,30, stopword_phrases=stop_phrases)
Trigram

[('federal reserve bank', 22),
 ('federal reserve system', 20),
 ('world war ii', 18),
 ('negative income tax', 17),
 ('increase quantity money', 15),
 ('rate monetary growth', 12),
 ('percent national income', 11),
 ('bank united state', 11),
 ('price wage control', 11),
 ('new york city', 10),
 ('public school system', 9),
 ('wall street journal', 8),
 ('income tax would', 8),
 ('percent per year', 8),
 ('state local government', 7),
 ('compulsory attendance law', 7),
 ('new york federal', 7),
 ('rapid increase quantity', 7),
 ('side effect cure', 7),
 ('adam smith invisible', 6),
 ('united state recent', 6),
 ('government united state', 6),
 ('interstate commerce commission', 6),
 ('central economic planning', 6),
 ('york federal reserve', 6),
 ('open market purchase', 6),
 ('increasing quantity money', 6),
 ('present welfare system', 6),
 ('national health service', 6),
 ('someone el money', 6)]

## 2. Get N-grams for the Series

In [67]:
# Open the txt of the book
from itertools import chain
with open(txt_series_path, 'rb') as file:
    text_string = file.read().decode('utf-8', errors='ignore')  # Ignora caracteres problemáticos
# Create chunks of text
chunks = text_string.split('.')
# Eliminate the withespaces

chunks = [chunk.strip() for chunk in chunks if chunk.strip()]

# Clean each chunk
clean_all = []
for chunk in chunks:
    clean_chunk = preprocess_text_video(chunk)
    clean_all.append(clean_chunk)

# Flatten the list of lists into a single list of tokens
clean_txt = list(chain.from_iterable(clean_all))

In [88]:
stop_phrases = {"nineteenth century", 'one another','new york', 'tide l turning', 'tide l turning', 'ever bigger government',
                'federal reserve board', "vol hello im", 'theyre gon na', 'join u next', 'free choose vol', 'choose vol hello', 
                'robert mckenzie welcome', 'hello im robert', 'im robert mckenzie', 'several people talking', 'hope youll join',
                'youll join u', 'harper library university', 'library university chicago', 'u next episode', 'im gon na', 
                'series free choose', 'week hope youll', 'film milton friedman', 'year five year', 'next episode free', 
                'episode free choose', 'mckenzie welcome university', 'welcome university chicago', 'new york city', 'gon na', 
                'milton friedman', 'dont think', 'go back', 'one thing', 'university chicago', 'dont know'
                }

# Bi-grama
bigram = top_k_ngrams(clean_txt,2,20, stopword_phrases=stop_phrases)
bigram_list= list(bigram)

# Bi-grama
trigram = top_k_ngrams(clean_txt,3,20, stopword_phrases=stop_phrases)
trigram_list= list(trigram)
####### Create latex table with the words #######
# LaTeX table header

latex_table = r"""
\begin{threeparttable}
\begin{tabular}{l l c l c}\\

\multicolumn{2}{c}{\textbf{Bigrams}}& & \multicolumn{2}{c}{\textbf{Trigrams}} \\
\cline{1-2} \cline{4-5} \textbf{Phrase} & \textbf{Count} & & \textbf{Phrase} & \textbf{Count}\\
\hline
\hline
"""

# Iterate over the list and add rows to the LaTeX table
for biagram, trigram in zip(bigram_list, trigram_list):
    phrase_b = biagram[0]
    count_b = biagram[1]

    phrase_t = trigram[0]
    count_t = trigram[1]
    
    latex_table += f"{phrase_b} & {count_b} & & {phrase_t} & {count_t} \\\\\n"

# LaTeX table footer
latex_table += r""" \hline
\hline
\end{tabular} \begin{tablenotes} 
\item{\footnotesize \tablenote} \end{tablenotes} \end{threeparttable}
"""

# Save the LaTeX table to a .txt file
with open("/Users/saracaicedo/Library/CloudStorage/OneDrive-Universidaddelosandes/2025-1/Tesis/output/tables/series_ngrams.tex", "w") as file:
    file.write(latex_table)

In [87]:
stop_phrases = {"nineteenth century", 'one another','new york', 'tide l turning', 'tide l turning', 'ever bigger government',
                'federal reserve board', "vol hello im", 'theyre gon na', 'join u next', 'free choose vol', 'choose vol hello', 
                'robert mckenzie welcome', 'hello im robert', 'im robert mckenzie', 'several people talking', 'hope youll join',
                'youll join u', 'harper library university', 'library university chicago', 'u next episode', 'im gon na', 
                'series free choose', 'week hope youll', 'film milton friedman', 'year five year', 'next episode free', 
                'episode free choose', 'mckenzie welcome university', 'welcome university chicago', 'new york city', 'gon na', 
                'milton friedman', 'dont think', 'go back', 'one thing', 'university chicago', 'dont know'
                }

# Tri-grama
Trigram = top_k_ngrams(clean_txt,2,20, stopword_phrases=stop_phrases)
Trigram

[('united state', 118),
 ('hong kong', 49),
 ('free market', 47),
 ('minimum wage', 47),
 ('free enterprise', 43),
 ('federal reserve', 38),
 ('income tax', 32),
 ('public school', 31),
 ('year ago', 29),
 ('government intervention', 29),
 ('labor union', 25),
 ('social security', 23),
 ('enterprise system', 22),
 ('negative income', 22),
 ('great depression', 21),
 ('welfare system', 20),
 ('free trade', 19),
 ('big government', 19),
 ('great deal', 18),
 ('trade union', 18)]